**Week 1 – Customer Sign-Up Behaviour & Data Quality Audit – Andrea Aguirre**

Task 1. Load & Clean the data

In [133]:
# Identify missing values, data types, and column structure
import pandas as pd

# Loading dataset
fileName = 'customer_signups.csv'
data = pd.read_csv(fileName)
print(data)

print("\n\nData types")
# Checking types
print(data.dtypes)

print("\n\nMissing data")
# Detecting missing data
print(data.isnull().sum())

print("\n\nData structure")
# Checking structure
data.info()

    customer_id                name                  email signup_date  \
0     CUST00000       Joshua Bryant                    NaN         NaN   
1     CUST00001      Nicole Stewart    nicole1@example.com    02-01-24   
2     CUST00002        Rachel Allen    rachel2@example.com    03-01-24   
3     CUST00003     Zachary Sanchez   zachary3@mailhub.org    04-01-24   
4     CUST00004                 NaN   matthew4@mailhub.org    05-01-24   
..          ...                 ...                    ...         ...   
295   CUST00295          Gary Smith     gary95@example.com    22-10-24   
296   CUST00296     Anthony Roberts  anthony96@mailhub.org    23-10-24   
297   CUST00297  Timothy Mclaughlin                    NaN    24-10-24   
298   CUST00298     Justin Mcintyre   justin98@mailhub.org    25-10-24   
299   CUST00299   Mr. Bruce Bridges      mr.99@example.com    26-10-24   

        source   region plan_selected marketing_opt_in age      gender  
0    Instagram      NaN         basic 

In [121]:
# Convert signup_date to datetime
data['signup_date'] = pd.to_datetime(data['signup_date'])
# Output
print(data['signup_date'])

0            NaT
1     2024-02-01
2     2024-03-01
3     2024-04-01
4     2024-05-01
         ...    
295   2024-10-22
296   2024-10-23
297   2024-10-24
298   2024-10-25
299   2024-10-26
Name: signup_date, Length: 300, dtype: datetime64[ns]


In [122]:
# Standardise inconsistent text values (plan_selected, gender, etc.)
data['plan_selected'] = data['plan_selected'].astype(str).str.lower().str.strip().replace({
    'pro': 'Pro', 
    'basic': 'Basic',
    'PREMIUM': 'Premium'
})
data['plan_selected'] = data['plan_selected'].str.capitalize()

# For gender column
data['gender'] = data['gender'].astype(str).str.lower().str.strip().replace({
    'MALE': 'Male', 
    'FEMALE': 'Female', 
    'male': 'Male', 
    'female': 'Female'
})
data['gender'] = data['gender'].str.capitalize()

# For 'marketing_opt_in'
data['marketing_opt_in'] = data['marketing_opt_in'].astype(str).str.lower().str.strip().replace({
    'Nil': 'None',
})
data['marketing_opt_in'] = data['marketing_opt_in'].str.capitalize()
# Output
print(data['plan_selected'])
print(data['gender'])
print(data['marketing_opt_in'])

0        Basic
1        Basic
2      Premium
3          Pro
4      Premium
        ...   
295    Premium
296      Basic
297      Basic
298    Premium
299    Premium
Name: plan_selected, Length: 300, dtype: object
0          Female
1            Male
2      Non-binary
3            Male
4           Other
          ...    
295           Nan
296        Female
297           Nan
298          Male
299          Male
Name: gender, Length: 300, dtype: object
0       No
1      Yes
2      Yes
3       No
4       No
      ... 
295    Yes
296    Yes
297    Yes
298     No
299    Yes
Name: marketing_opt_in, Length: 300, dtype: object


In [123]:
# Remove duplicate rows based on customer_id 
data_before = data.shape[0]
data = data.drop_duplicates(subset='customer_id')
data_no_dupes = data_before- data.shape[0]
print(data_no_dupes)

1


In [124]:
# Handle missing values (e.g., region, email, age) 
data_filled = data.copy()
data_filled['region'] = data_filled['region'].fillna('Unknown')
data_filled['age'] = pd.to_numeric(data_filled['age'], errors='coerce')
data_filled['age'] = data_filled['age'].fillna(data_filled['age'].median())
data_filled = data_filled.dropna(subset=['email'])

data_dropped = data.copy()
data_dropped['age'] = pd.to_numeric(data_dropped['age'], errors='coerce')
data_dropped = data_dropped.dropna()

data_interp = data.copy()
data_interp['age'] = pd.to_numeric(data_interp['age'], errors='coerce')

# Infer better dtypes before interpolation (fixes FutureWarning)
data_interp = data_interp.infer_objects()

# Interpolate only the 'age' column to avoid PeriodArray error
data_interp['age'] = data_interp['age'].interpolate()

print("Replaced:\n", data_filled)
print("Removed:\n", data_dropped)
print("Interpolate:\n", data_interp)

Replaced:
     customer_id                name                  email signup_date  \
1     CUST00001      Nicole Stewart    nicole1@example.com  2024-02-01   
2     CUST00002        Rachel Allen    rachel2@example.com  2024-03-01   
3     CUST00003     Zachary Sanchez   zachary3@mailhub.org  2024-04-01   
4     CUST00004                 NaN   matthew4@mailhub.org  2024-05-01   
5     CUST00005       John Gonzales      john5@mailhub.org  2024-06-01   
..          ...                 ...                    ...         ...   
294   CUST00294  Mrs. Jessica Smith     mrs.94@example.com  2024-10-21   
295   CUST00295          Gary Smith     gary95@example.com  2024-10-22   
296   CUST00296     Anthony Roberts  anthony96@mailhub.org  2024-10-23   
298   CUST00298     Justin Mcintyre   justin98@mailhub.org  2024-10-25   
299   CUST00299   Mr. Bruce Bridges      mr.99@example.com  2024-10-26   

       source   region plan_selected marketing_opt_in   age      gender  
1    LinkedIn     West    

Task 2. Data Quality Summary

In [125]:
# Count of missing values per column 
missing_count = data.isnull().sum()
print("Missing Values:\n", missing_count)

Missing Values:
 customer_id          1
name                 9
email               34
signup_date          6
source               9
region              30
plan_selected        0
marketing_opt_in     0
age                 12
gender               0
dtype: int64


In [126]:
# % of missing values 
missing_percent = data.isnull().mean() * 100
print("\nMissing Values (%):\n", missing_percent)


Missing Values (%):
 customer_id          0.334448
name                 3.010033
email               11.371237
signup_date          2.006689
source               3.010033
region              10.033445
plan_selected        0.000000
marketing_opt_in     0.000000
age                  4.013378
gender               0.000000
dtype: float64


In [127]:
# Number of duplicates removed 
print("\nDuplicates removed", data_no_dupes)


Duplicates removed 1


In [128]:
# Mention inconsistent category values corrected (e.g., PRO → Pro)
print("\nCorrected categories:")
print("plan_selected: 'pro' to 'Pro', 'basic' to 'Basic'")
print("gender: 'MALE'/'male' to 'Male', 'FEMALE'/'female' to 'Female'")
print("marketing_opt_in: 'Nil' to 'None', 'yes' to 'Yes', 'no' to 'No'")


Corrected categories:
plan_selected: 'pro' to 'Pro', 'basic' to 'Basic'
gender: 'MALE'/'male' to 'Male', 'FEMALE'/'female' to 'Female'
marketing_opt_in: 'Nil' to 'None', 'yes' to 'Yes', 'no' to 'No'


Task 3. Summary Outputs

In [129]:
# Sign-ups per week

# Making sure that signup_date is datetime
data['signup_date'] = pd.to_datetime(data['signup_date'])

# Extracting to a week period to count how many customers joined in each week
data['signup_week'] = data['signup_date'].dt.to_period('W')
signups_per_week = data.groupby('signup_week')['customer_id'].count()

# Output
print("\nSign-ups per week by signup_date):\n", signups_per_week)


Sign-ups per week by signup_date):
 signup_week
2024-01-01/2024-01-07    6
2024-01-08/2024-01-14    5
2024-01-15/2024-01-21    7
2024-01-22/2024-01-28    7
2024-01-29/2024-02-04    8
2024-02-05/2024-02-11    6
2024-02-12/2024-02-18    6
2024-02-19/2024-02-25    7
2024-02-26/2024-03-03    7
2024-03-04/2024-03-10    7
2024-03-11/2024-03-17    5
2024-03-18/2024-03-24    6
2024-03-25/2024-03-31    6
2024-04-01/2024-04-07    7
2024-04-08/2024-04-14    5
2024-04-15/2024-04-21    7
2024-04-22/2024-04-28    7
2024-04-29/2024-05-05    6
2024-05-06/2024-05-12    4
2024-05-13/2024-05-19    7
2024-05-20/2024-05-26    7
2024-05-27/2024-06-02    7
2024-06-03/2024-06-09    7
2024-06-10/2024-06-16    5
2024-06-17/2024-06-23    7
2024-06-24/2024-06-30    7
2024-07-01/2024-07-07    7
2024-07-08/2024-07-14    5
2024-07-15/2024-07-21    6
2024-07-22/2024-07-28    7
2024-07-29/2024-08-04    7
2024-08-05/2024-08-11    5
2024-08-12/2024-08-18    6
2024-08-19/2024-08-25    7
2024-08-26/2024-09-01    7
2024-0

In [130]:
# Sign-ups by source, region, plan_selected

# Counting how many customers have signed up by source, region and plan selected
signups_by_group = data.groupby(['source', 'region', 'plan_selected'])['customer_id'].count()
print("\nSign-ups by source, region, and plan_selected:\n", signups_by_group)


Sign-ups by source, region, and plan_selected:
 source   region   plan_selected
??       Central  Pro              1
         East     Pro              1
         South    Pro              1
         West     Basic            1
                  Premium          1
                                  ..
YouTube  South    Pro              2
         West     Basic            1
                  Nan              1
                  Premium          5
                  Pro              1
Name: customer_id, Length: 100, dtype: int64


In [131]:
# Marketing opt-in counts by gender
opt_in_by_gender = data.groupby('gender')['marketing_opt_in'].value_counts()
print("\nMarketing opt-in counts by gender:\n", opt_in_by_gender)


Marketing opt-in counts by gender:
 gender      marketing_opt_in
123         No                   3
            Yes                  3
Female      No                  47
            Yes                 44
            Nan                  1
Male        No                  50
            Yes                 38
            Nan                  3
            Nil                  1
Nan         No                   4
            Yes                  4
Non-binary  No                  20
            Yes                 19
            Nan                  3
Other       No                  32
            Yes                 24
            Nan                  3
Name: count, dtype: int64


In [132]:
# Age summary
# Convert to numeric
age_numeric = pd.to_numeric(data['age'], errors='coerce')  

age_summary = age_numeric.agg(['min', 'max', 'mean', 'median'])
age_null_count = age_numeric.isnull().sum()

print("\nAge Summary:\n", age_summary)
print("Null count in age:", age_null_count)


Age Summary:
 min        21.000
max       206.000
mean       36.175
median     34.000
Name: age, dtype: float64
Null count in age: 19
